# Import Libraries

In [2]:
# Pandas is used for the dataframe
# Pyscopg2 is used for connecting Python with Postgres
import pandas as pd
import psycopg2 as pg

# Extract

In [3]:
# pd.read_excel to read the worksheet convert it into dataframe

with pd.ExcelFile("dataset.xlsx", engine='openpyxl') as xls:
    df_transaction = pd.read_excel(xls,sheet_name='transaction')
    df_transaction_event = pd.read_excel(xls,sheet_name='transaction_event')
    df_agent = pd.read_excel(xls,sheet_name='agent')


c:\Users\dwika\Miniconda3\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
df_transaction

,id,agent_id,payment_status,total_premi,product_id,registraton_date
0,103535,31569,PAID,13127500.0,6,2021-12-01 00:26:20
1,103539,36808,PAID,6400000.0,6,2021-12-01 00:43:38
2,103547,7056,PAID,4883800.0,6,2021-12-01 01:31:33
3,103555,26928,PAID,5505480.0,6,2021-12-01 01:56:24
4,103559,37612,PAID,11825000.0,6,2021-12-01 01:58:07
...,...,...,...,...,...,...
21256,151587,44783,WAITING,4280800.0,6,2022-03-31 08:25:01
21257,151591,39862,WAITING,5406000.0,6,2022-03-31 08:30:15
21258,151599,45977,PAID,20000.0,8,2022-03-31 08:39:27
21259,151607,28025,PAID,20000.0,8,2022-03-31 08:46:46


In [5]:
df_transaction_event

,id,transaction_id,status,created_at
0,365212,103532,WAITING,2021-12-01 00:11:52
1,365760,103532,ON_PROCESS,2021-12-01 04:12:28
2,366517,103532,APPROVED,2021-12-01 07:52:53
3,365215,103533,WAITING,2021-12-01 00:16:19
4,365799,103533,ON_PROCESS,2021-12-01 04:19:49
...,...,...,...,...
49319,638309,151606,WAITING,2022-03-31 08:46:05
49320,638312,151607,WAITING,2022-03-31 08:46:46
49321,638336,151608,WAITING,2022-03-31 08:49:47
49322,638350,151611,WAITING,2022-03-31 08:51:20


In [6]:
df_agent

,agent_id,join_date,tier_id
0,5,2019-09-06 20:48:00,88
1,9,2019-09-08 19:38:23,88
2,13,2019-09-09 15:35:16,88
3,17,2019-09-09 20:46:45,72
4,21,2019-09-10 15:01:23,88
...,...,...,...
37951,45972,2022-03-31 11:48:54,88
37952,45976,2022-03-31 12:22:41,88
37953,45980,2022-03-31 12:49:01,88
37954,45984,2022-03-31 14:02:20,88


# Transformation

## Data Cleaning

### Transaction

In [7]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21261 entries, 0 to 21260
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                21261 non-null  int64  
 1   agent_id          21261 non-null  int64  
 2   payment_status    21259 non-null  object 
 3   total_premi       21261 non-null  float64
 4   product_id        21261 non-null  int64  
 5   registraton_date  21261 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 996.7+ KB


In [8]:
# Minor spelling error on registraton_date, change to registration_date
df_transaction = df_transaction.rename(columns={'registraton_date':'registration_date'})

In [9]:
# df_transaction contains 9 duplicates data
df_transaction.duplicated().sum()

np.int64(9)

In [10]:
df_transaction[df_transaction.duplicated(keep=False)]

,id,agent_id,payment_status,total_premi,product_id,registration_date
447,107775,36447,WAITING,5473150.0,6,2021-12-10 09:59:17
448,107775,36447,WAITING,5473150.0,6,2021-12-10 09:59:17
657,109597,17722,PAID,3410000.0,6,2021-12-15 14:33:13
658,109597,17722,PAID,3410000.0,6,2021-12-15 14:33:13
659,109597,17722,PAID,3410000.0,6,2021-12-15 14:33:13
3900,108024,7225,NaN,6591600.0,6,2021-12-11 09:42:50
3901,108024,7225,NaN,6591600.0,6,2021-12-11 09:42:50
7689,149422,40583,WAITING,5139000.0,6,2022-03-28 08:01:15
7690,149422,40583,WAITING,5139000.0,6,2022-03-28 08:01:15
10881,105884,25388,PAID,5540000.0,6,2021-12-07 02:05:37


In [11]:
# drop to erase the duplicate
df_transaction = df_transaction.drop_duplicates()
df_transaction

,id,agent_id,payment_status,total_premi,product_id,registration_date
0,103535,31569,PAID,13127500.0,6,2021-12-01 00:26:20
1,103539,36808,PAID,6400000.0,6,2021-12-01 00:43:38
2,103547,7056,PAID,4883800.0,6,2021-12-01 01:31:33
3,103555,26928,PAID,5505480.0,6,2021-12-01 01:56:24
4,103559,37612,PAID,11825000.0,6,2021-12-01 01:58:07
...,...,...,...,...,...,...
21256,151587,44783,WAITING,4280800.0,6,2022-03-31 08:25:01
21257,151591,39862,WAITING,5406000.0,6,2022-03-31 08:30:15
21258,151599,45977,PAID,20000.0,8,2022-03-31 08:39:27
21259,151607,28025,PAID,20000.0,8,2022-03-31 08:46:46


In [12]:
# df_transaction has 1 row with missing value for payment_status
df_transaction.isna().sum()

id                   0
agent_id             0
payment_status       1
total_premi          0
product_id           0
registration_date    0
dtype: int64

In [13]:
df_transaction[df_transaction['payment_status'].isna()]

,id,agent_id,payment_status,total_premi,product_id,registration_date
3900,108024,7225,NaN,6591600.0,6,2021-12-11 09:42:50


In [14]:
df_transaction_event[df_transaction_event['transaction_id'] == 108024]

,id,transaction_id,status,created_at
4671,389467,108024,WAITING,2021-12-11 09:42:50


In [15]:
df_transaction[df_transaction['payment_status'].isna()]

,id,agent_id,payment_status,total_premi,product_id,registration_date
3900,108024,7225,NaN,6591600.0,6,2021-12-11 09:42:50


The missing value happened in transaction id 108024. This transaction is currently at WAITING status. Not only that but the registration_date and created_at time are the same. Therefore, it can be assume that the missing value is WAITING

In [16]:
df_transaction.loc[df_transaction['id'] == 108024, 'payment_status'] = 'WAITING'

In [17]:
df_transaction.loc[df_transaction['id'] == 108024]

,id,agent_id,payment_status,total_premi,product_id,registration_date
3900,108024,7225,WAITING,6591600.0,6,2021-12-11 09:42:50


In [18]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21252 entries, 0 to 21260
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 21252 non-null  int64  
 1   agent_id           21252 non-null  int64  
 2   payment_status     21252 non-null  object 
 3   total_premi        21252 non-null  float64
 4   product_id         21252 non-null  int64  
 5   registration_date  21252 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 1.1+ MB


regeistration_date is still in object

### Transaction Event

In [19]:
df_transaction_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49324 entries, 0 to 49323
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              49324 non-null  int64         
 1   transaction_id  49324 non-null  int64         
 2   status          49324 non-null  object        
 3   created_at      49324 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 1.5+ MB


In [20]:
df_transaction_event.duplicated().sum()

np.int64(0)

transaction_event looks clean. no further alteration

### Agent

In [21]:
df_agent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37956 entries, 0 to 37955
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   agent_id   37956 non-null  int64         
 1   join_date  37956 non-null  datetime64[ns]
 2   tier_id    37956 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 889.7 KB


In [22]:
df_agent.duplicated().sum()

np.int64(0)

In [23]:
df_agent.loc[df_agent['agent_id'] == 5935]

,agent_id,join_date,tier_id


In [24]:
df_transaction.loc[df_transaction['agent_id'] == 5935]

,id,agent_id,payment_status,total_premi,product_id,registration_date
398,107327,5935,WAITING,1292500.0,6,2021-12-09 11:11:01
1573,117732,5935,WAITING,4201470.0,6,2022-01-07 06:51:28
2090,122955,5935,WAITING,4371570.0,6,2022-01-21 12:57:22
3129,131192,5935,WAITING,1250000.0,6,2022-02-12 09:04:02
3358,106842,5935,WAITING,4322500.0,6,2021-12-08 11:03:53
4076,109616,5935,WAITING,20000.0,8,2021-12-15 15:40:38
5473,117110,5935,PAID,20000.0,8,2022-01-05 15:26:37
5475,117122,5935,PAID,20000.0,8,2022-01-05 15:45:41
5891,140136,5935,WAITING,3275220.0,6,2022-03-07 08:52:14
6077,141931,5935,WAITING,2022500.0,12,2022-03-10 12:33:34


df_transaction has the agent_id of 5935 but df_agent itself doesn't. 

In [25]:
count_missing_5935 = len(df_transaction.loc[df_transaction['agent_id'] == 5935])
print(count_missing_5935)

41


In [26]:
df_agent.loc[df_agent['agent_id'].astype('str').str.contains('5935')]

,agent_id,join_date,tier_id
15421,45935,2022-03-30 19:20:33,88
23406,15935,2021-04-16 14:27:44,88
28891,35935,2021-10-31 15:08:15,88
33712,25935,2021-07-09 15:26:20,88


Missing value of 5935 could be a missed input of 45935, 15935, 35935, 25935. Therefore the 5935 id will be replace with -1 as a flag.

In [27]:
# To find agent_id that present in transaction_fact but not in agent dimension
bad_agents = df_transaction[
    ~df_transaction['agent_id'].isin(df_agent['agent_id'])
]
bad_agents['agent_id'].unique()

array([ 5935, 39466, 12104, 44889, 39556,  7551,  4131, 39316, 39995,
       34448, 18123,   588, 37789, 17401,   169])

In [28]:
# replace the agent_id that present in transaction_fact but not in agent dimension with -1
df_transaction.loc[df_transaction['agent_id'].isin(bad_agents['agent_id']),'agent_id'] = -1

In [29]:
df_transaction.loc[df_transaction['agent_id'] == -1]

,id,agent_id,payment_status,total_premi,product_id,registration_date
398,107327,-1,WAITING,1292500.0,6,2021-12-09 11:11:01
1474,116771,-1,WAITING,166000.0,12,2022-01-05 04:33:41
1497,117035,-1,WAITING,4036500.0,4,2022-01-05 11:02:51
1573,117732,-1,WAITING,4201470.0,6,2022-01-07 06:51:28
2090,122955,-1,WAITING,4371570.0,6,2022-01-21 12:57:22
...,...,...,...,...,...,...
20778,147453,-1,WAITING,12937500.0,6,2022-03-23 15:23:08
20781,147465,-1,WAITING,5430000.0,6,2022-03-23 16:19:59
20831,147965,-1,WAITING,4883800.0,6,2022-03-24 14:38:44
21058,149927,-1,PAID,20000.0,8,2022-03-29 03:56:02


### Total Premi

In [30]:
df_transaction['total_premi'].value_counts()

total_premi
20000.0      4742
150000.0      145
44400.0       142
75000.0        82
4481800.0      67
             ... 
4902540.0       1
3410880.0       1
6827500.0       1
3683765.0       1
6925956.0       1
Name: count, Length: 10464, dtype: int64

In [31]:
df_transaction.loc[df_transaction['total_premi'].isnull()]

,id,agent_id,payment_status,total_premi,product_id,registration_date


In [32]:
df_transaction.loc[df_transaction['total_premi'] < 0]

,id,agent_id,payment_status,total_premi,product_id,registration_date


total_premi looks clean. no alteration.

In [33]:
df_transaction[df_transaction['registration_date'].str.contains('/' ,na=False)]

,id,agent_id,payment_status,total_premi,product_id,registration_date
3114,131056,42539,PAID,5351290.0,6,2/29/2022 16:43:12


registration_date datatype of object was cause by imposible date as there is no day 29 in February 2022.

In [34]:
df_transaction_event[df_transaction_event['transaction_id'] == 131056]

,id,transaction_id,status,created_at
28690,521689,131056,WAITING,2022-02-11 16:43:12
28691,522098,131056,ON_PROCESS,2022-02-12 05:55:08
28692,532385,131056,APPROVED,2022-02-17 05:10:38


transcation_id 131056 has already been approved at 	2022-02-17 05:10:38 

In [35]:
# pd.to_datetime to change object into datetime
# errors = 'coerce' to change error into N/A
df_transaction['registration_date'] = pd.to_datetime(df_transaction['registration_date'], errors='coerce')
df_transaction.loc[df_transaction['id'] == 131056]

,id,agent_id,payment_status,total_premi,product_id,registration_date
3114,131056,42539,PAID,5351290.0,6,NaT


In [36]:
df_transaction['registration_date'] = df_transaction['registration_date'].fillna(pd.Timestamp('2022-02-17 05:10:38'))
df_transaction.loc[df_transaction['id'] == 131056]

,id,agent_id,payment_status,total_premi,product_id,registration_date
3114,131056,42539,PAID,5351290.0,6,2022-02-17 05:10:38


In [37]:
# dt.month to extract month from registration_date
df_transaction.loc[:, 'registration_month'] = df_transaction['registration_date'].dt.month
df_transaction.loc[:, 'registration_year'] = df_transaction['registration_date'].dt.year
df_transaction.loc[:, 'registration_quater'] = df_transaction['registration_date'].dt.quarter
df_transaction.loc[:, 'registration_day'] = df_transaction['registration_date'].dt.day
df_transaction

,id,agent_id,payment_status,total_premi,product_id,registration_date,registration_month,registration_year,registration_quater,registration_day
0,103535,31569,PAID,13127500.0,6,2021-12-01 00:26:20,12,2021,4,1
1,103539,36808,PAID,6400000.0,6,2021-12-01 00:43:38,12,2021,4,1
2,103547,7056,PAID,4883800.0,6,2021-12-01 01:31:33,12,2021,4,1
3,103555,26928,PAID,5505480.0,6,2021-12-01 01:56:24,12,2021,4,1
4,103559,37612,PAID,11825000.0,6,2021-12-01 01:58:07,12,2021,4,1
...,...,...,...,...,...,...,...,...,...,...
21256,151587,44783,WAITING,4280800.0,6,2022-03-31 08:25:01,3,2022,1,31
21257,151591,39862,WAITING,5406000.0,6,2022-03-31 08:30:15,3,2022,1,31
21258,151599,45977,PAID,20000.0,8,2022-03-31 08:39:27,3,2022,1,31
21259,151607,28025,PAID,20000.0,8,2022-03-31 08:46:46,3,2022,1,31


## Dataframe for Dimension & Facts

In [38]:
df_agent_dim = df_agent.copy()
df_agent_dim

,agent_id,join_date,tier_id
0,5,2019-09-06 20:48:00,88
1,9,2019-09-08 19:38:23,88
2,13,2019-09-09 15:35:16,88
3,17,2019-09-09 20:46:45,72
4,21,2019-09-10 15:01:23,88
...,...,...,...
37951,45972,2022-03-31 11:48:54,88
37952,45976,2022-03-31 12:22:41,88
37953,45980,2022-03-31 12:49:01,88
37954,45984,2022-03-31 14:02:20,88


In [39]:
df_dim_date = pd.concat([df_agent['join_date'], df_transaction['registration_date'], df_transaction_event['created_at']], ignore_index=True).to_frame('full_date')
df_dim_date = df_dim_date.drop_duplicates()
df_dim_date = df_dim_date.sort_values(by='full_date')
df_dim_date = df_dim_date.reset_index(drop=True)

df_dim_date.loc[:, 'quarter'] = df_dim_date['full_date'].dt.quarter
df_dim_date.loc[:, 'year'] = df_dim_date['full_date'].dt.year
df_dim_date.loc[:, 'month'] = df_dim_date['full_date'].dt.month
df_dim_date.loc[:, 'day'] = df_dim_date['full_date'].dt.day



df_dim_date

,full_date,quarter,year,month,day
0,2019-09-06 20:48:00,3,2019,9,6
1,2019-09-06 21:35:54,3,2019,9,6
2,2019-09-08 19:38:23,3,2019,9,8
3,2019-09-08 20:46:18,3,2019,9,8
4,2019-09-08 22:00:49,3,2019,9,8
...,...,...,...,...,...
88820,2022-03-31 14:53:40,1,2022,3,31
88821,2022-03-31 15:04:52,1,2022,3,31
88822,2022-03-31 15:06:15,1,2022,3,31
88823,2022-03-31 15:06:45,1,2022,3,31


In [40]:
df_dim_status = pd.concat([df_transaction['payment_status'],df_transaction_event['status']],ignore_index=True).to_frame('status_name')
df_dim_status = df_dim_status.drop_duplicates()
df_dim_status = df_dim_status.reset_index(drop=True)

df_dim_status['status_id'] = df_dim_status.index + 1
df_dim_status = df_dim_status[['status_id','status_name']]
df_dim_status


,status_id,status_name
0,1,PAID
1,2,WAITING
2,3,ON_PROCESS
3,4,APPROVED


In [41]:
status_lookup = dict(zip(df_dim_status['status_name'], df_dim_status['status_id']))

df_facts_transaction = df_transaction.copy()
df_facts_transaction['status_id'] = df_facts_transaction['payment_status'].map(status_lookup)
df_facts_transaction = df_facts_transaction[['id','agent_id','product_id','status_id','registration_date','total_premi']]

df_facts_transaction

,id,agent_id,product_id,status_id,registration_date,total_premi
0,103535,31569,6,1,2021-12-01 00:26:20,13127500.0
1,103539,36808,6,1,2021-12-01 00:43:38,6400000.0
2,103547,7056,6,1,2021-12-01 01:31:33,4883800.0
3,103555,26928,6,1,2021-12-01 01:56:24,5505480.0
4,103559,37612,6,1,2021-12-01 01:58:07,11825000.0
...,...,...,...,...,...,...
21256,151587,44783,6,2,2022-03-31 08:25:01,4280800.0
21257,151591,39862,6,2,2022-03-31 08:30:15,5406000.0
21258,151599,45977,8,1,2022-03-31 08:39:27,20000.0
21259,151607,28025,8,1,2022-03-31 08:46:46,20000.0


In [41]:
df_facts_transaction_event = df_transaction_event.copy()
df_facts_transaction_event['status_id'] = df_facts_transaction_event['status'].map(status_lookup)
df_facts_transaction_event = df_facts_transaction_event[['id','transaction_id','status_id','created_at']]
df_facts_transaction_event

,id,transaction_id,status_id,created_at
0,365212,103532,2,2021-12-01 00:11:52
1,365760,103532,3,2021-12-01 04:12:28
2,366517,103532,4,2021-12-01 07:52:53
3,365215,103533,2,2021-12-01 00:16:19
4,365799,103533,3,2021-12-01 04:19:49
...,...,...,...,...
49319,638309,151606,2,2022-03-31 08:46:05
49320,638312,151607,2,2022-03-31 08:46:46
49321,638336,151608,2,2022-03-31 08:49:47
49322,638350,151611,2,2022-03-31 08:51:20


# Load

In [42]:
# Setting up the connection between python and postgresql
pgConn = pg.connect(
    dbname = "qoala_test",
    user = "postgres",
    password = "postgres",
    host = "127.0.0.1",
    port = "5432"
)

# DDL of Data Warehouse
cur = pgConn.cursor()
cur.execute(
    """
    DROP TABLE IF EXISTS facts_transaction CASCADE;
    DROP TABLE IF EXISTS facts_transaction_event CASCADE;
    DROP TABLE IF EXISTS dim_agent CASCADE;
    DROP TABLE IF EXISTS dim_date CASCADE;
    DROP TABLE IF EXISTS dim_transaction_status CASCADE;

    CREATE TABLE IF NOT EXISTS dim_agent(
        agent_id INT PRIMARY KEY,
        tier_id INT,
        join_date TIMESTAMP
    );

    CREATE TABLE IF NOT EXISTS dim_date(
        full_date TIMESTAMP PRIMARY KEY,
        quarter INT,
        year INT, 
        month INT,
        day INT
    );

    CREATE TABLE IF NOT EXISTS dim_transaction_status(
        id INT PRIMARY KEY,
        status_name VARCHAR(20)
    );

    CREATE TABLE IF NOT EXISTS facts_transaction(
        id INT PRIMARY KEY,
        agent_id INT,
        product_id INT,
        status_id INT,
        registration_date TIMESTAMP,
        total_premi FLOAT,
        CONSTRAINT fk_agent FOREIGN KEY (agent_id) REFERENCES dim_agent(agent_id),
        CONSTRAINT fk_date FOREIGN KEY (registration_date) REFERENCES dim_date(full_date),
        CONSTRAINT fk_status FOREIGN KEY(status_id) REFERENCES dim_transaction_status(id)
    );

    CREATE TABLE IF NOT EXISTS facts_transaction_event(
        id INT PRIMARY KEY,
        transaction_id INT,
        status_id INT,
        created_at TIMESTAMP,
        CONSTRAINT fk_transaction FOREIGN KEY (transaction_id) REFERENCES facts_transaction(id),
        CONSTRAINT fk_date FOREIGN KEY (created_at) REFERENCES dim_date(full_date),
        CONSTRAINT fk_status FOREIGN KEY (status_id) REFERENCES dim_transaction_status(id)
    );
"""
)

pgConn.commit()
pgConn.close()

In [43]:
pgConn = pg.connect(
    dbname = "qoala_test",
    user = "postgres",
    password = "postgres",
    host = "127.0.0.1",
    port = "5432"
)
# Insert a row with agent_id having -1 as a flag
curr = pgConn.cursor()
curr.execute(
    """
    INSERT INTO dim_agent (agent_id, join_date, tier_id)
    VALUES (-1, NULL, NULL);
"""
)

pgConn.commit()
pgConn.close()

In [44]:
# Script to insert the dataframe into the facts and dimension table
pgConn = pg.connect(
    dbname = "qoala_test",
    user = "postgres",
    password = "postgres",
    host = "127.0.0.1",
    port = "5432"
)

curr = pgConn.cursor()

curr.executemany(
    """
    INSERT INTO dim_agent(agent_id,join_date, tier_id)
    VALUES (%s, %s, %s)
""", df_agent_dim.values.tolist()
)

curr.executemany(
    """
    INSERT INTO dim_date(full_date, quarter, year, month, day)
    VALUES (%s, %s, %s, %s, %s)
""", df_dim_date.values.tolist()
)

curr.executemany(
    """
    INSERT INTO dim_transaction_status(id, status_name)
    VALUES (%s, %s)
""", df_dim_status.values.tolist()
)

curr.executemany(
    """
    INSERT INTO facts_transaction(id, agent_id, product_id, status_id, registration_date, total_premi)
    VALUES (%s, %s, %s, %s, %s, %s)
""", df_facts_transaction.values.tolist()
)

curr.executemany(
    """
    INSERT INTO facts_transaction_event(id, transaction_id, status_id, created_at)
    VALUES (%s, %s, %s, %s)
""", df_facts_transaction_event.values.tolist()
)
pgConn.commit()
pgConn.close()
